<a href="https://colab.research.google.com/github/Chirag-Kathuria-009/FEVER_FACT_CHECKER/blob/master/NLP_Fact_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install wikipedia
!pip install -U datasets
from datasets import load_dataset

# Load FEVER dataset which includes Wikipedia pages
#fever_dataset = load_dataset("fever", "wiki_pages",trust_remote_code=True)
fever_dataset = load_dataset('fever', 'wiki_pages', streaming=True,trust_remote_code=True)
# Load fever dataset for information about claims data
claims_data = load_dataset("fever", "v1.0",split='train',trust_remote_code=True)

claims_data_validation = load_dataset("fever", "v1.0",split='labelled_dev',trust_remote_code=True)
#claims_data_validation

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:

from itertools import groupby
distinct_sentenceids = {k for k, _ in groupby(sorted(d['evidence_wiki_url'] for d in claims_data))}
validation_sentenceids = {k for k, _ in groupby(sorted(d['evidence_wiki_url'] for d in claims_data_validation))}
distinct_sentenceids.update(validation_sentenceids)

print(distinct_sentenceids)
#print(claims_data[0])

{'', 'Emotive_-LRB-album-RRB-', 'George_Lucas', 'Get_on_Up_-LRB-film-RRB-', "Ocean's_Eight", 'Naanu_Naane', 'Robert_Redford', 'Days_of_Heaven', 'America_Ferrera', 'Aware_Records', 'Capital_G', 'Up_in_the_Air_-LRB-2009_film-RRB-', 'Allen_Ginsberg', 'Anthology_series', 'Tinker_Tailor_Soldier_Spy_-LRB-film-RRB-', 'Katie_Holmes', 'Stratford,_London', 'The_Secret_Life_of_Walter_Mitty_-LRB-2013_film-RRB-', 'Elias_Koteas', 'Gracepoint', 'In_the_Loop', 'Lynda_Obst_Productions', 'Neo_Ned', 'Heath_Ledger', 'Vantage_Point_-LRB-film-RRB-', 'Circus_-LRB-Britney_Spears_album-RRB-', 'Creative_work', 'List_of_cities_in_Denmark_by_population', 'List_of_Governors_of_Arizona', 'I_Now_Pronounce_You_Chuck_&_Larry', 'Art_Institute_of_Chicago', 'Deewangee', 'The_Burning_Plain', 'FIFA_U-20_World_Cup', 'Tony_Bill', 'King_of_the_Ring_-LRB-2002-RRB-', 'Lynette_Scavo', 'Terra_Nova_-LRB-TV_series-RRB-', 'Digging_to_China', 'E-commerce', 'Lizzy_Caplan', 'Shueisha', 'Atomic_number', 'Octopus', 'Leland_Orser', 'Vampi

In [ ]:
#distinct_sentenceids.remove('')

import tqdm
import unicodedata

pages = fever_dataset['wikipedia_pages']
filtered_pages = []
for page in tqdm.tqdm(pages):
  page_id = str(page.get("id"))
  if page_id in distinct_sentenceids and page not in filtered_pages:
    filtered_pages.append(page)

  if len(filtered_pages) == len(distinct_sentenceids):
        break

print(len(filtered_pages))
#Remind me of the roadmap for my fact checker project.”



In [4]:
#print(len(filtered_pages))
#print(len(distinct_sentenceids))
#print(filtered_pages[:10])
from google.colab import drive
import pickle
drive.mount('/content/drive')


# Assuming `filtered_pages` is your list
'''with open("/content/drive/MyDrive/filtered_pages.pkl", "wb") as f:
    pickle.dump(filtered_pages, f)
print("Saved filtered_pages to filtered_pages.pkl")'''

with open("/content/drive/MyDrive/filtered_pages.pkl", "rb") as f:
    filtered_pages = pickle.load(f)
print("Loaded filtered_pages from filtered_pages.pkl")
#print(d[0])
print(claims_data[:10])
print(len(filtered_pages))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded filtered_pages from filtered_pages.pkl
{'id': [75397, 75397, 150448, 150448, 214861, 156709, 83235, 129629, 129629, 149579], 'label': ['SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'REFUTES', 'NOT ENOUGH INFO', 'SUPPORTS', 'SUPPORTS', 'NOT ENOUGH INFO'], 'claim': ['Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.', 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.', 'Roman Atwood is a content creator.', 'Roman Atwood is a content creator.', 'History of art includes architecture, dance, sculpture, music, painting, poetry literature, theatre, narrative, film, photography and graphic arts.', 'Adrienne Bailon is an accountant.', 'System of a Down briefly disbanded in limbo.', 'Homeland is an American television spy thriller based on the Israeli television series Prisoners of War.', 'Homeland is an American televi

In [5]:
def indexing_wiki_pages():
  wiki_index = {}
  error_pages = set()
  for page in filtered_pages:
    page_id = page['id']
    lines = page['lines'].split('\n')
    wiki_index[page_id] = {}
    for line in lines:
      if line.strip():
        try:
          sid, sentence = line.split("\t",1)
          wiki_index[page_id][int(sid)] = sentence

        except ValueError as error:
          error_pages.add(page_id)
          continue

  return wiki_index




In [6]:
from datasets import Dataset
def mapping_wiki_pages_sentence():
  indexed_sentence_pages = indexing_wiki_pages()
  missing_keys = set()
  corr_sentences = []
  for claim in claims_data:
    page_id = claim['evidence_wiki_url']
    sentence_id = claim['evidence_sentence_id']
    sentence=''
    if(sentence_id !=-1):

      try:
        value = indexed_sentence_pages.get(page_id)
        if value is not None:
          sentence = indexed_sentence_pages[page_id][sentence_id]
          #claim['corr_sentence'] = sentence
          #print('Sentences',claim['corr_sentence'])
        else:
          missing_keys.add(page_id)


      except ValueError as error:
        sentence = None
        print('Error with page_id',page_id+str(error))
        continue
    corr_sentences.append(sentence)



  if len(corr_sentences) != len(claims_data):
    print(f"Warning: Length mismatch! corr_sentences length: {len(corr_sentences)} vs claims_data length: {len(claims_data)}")

  if isinstance(claims_data, Dataset):
    updated_claims_data = claims_data.add_column('corr_sentence', corr_sentences)
  else:
    print("Error: claims_data is not of type datasets.Dataset")
  return updated_claims_data,missing_keys

#print(len(indexed_sentence_pages))



In [7]:
from datasets import Dataset
def mapping_wiki_pages_sentence_validation():
  indexed_sentence_pages = indexing_wiki_pages()
  missing_keys = set()
  corr_sentences = []
  for claim in claims_data_validation:
    page_id = claim['evidence_wiki_url']
    sentence_id = claim['evidence_sentence_id']
    sentence=''
    if(sentence_id !=-1):

      try:
        value = indexed_sentence_pages.get(page_id)
        if value is not None:
          sentence = indexed_sentence_pages[page_id][sentence_id]
          #claim['corr_sentence'] = sentence
          #print('Sentences',claim['corr_sentence'])
        else:
          missing_keys.add(page_id)


      except ValueError as error:
        sentence = None
        print('Error with page_id',page_id+str(error))
        continue
    corr_sentences.append(sentence)



  if len(corr_sentences) != len(claims_data_validation):
    print(f"Warning: Length mismatch! corr_sentences length: {len(corr_sentences)} vs claims_data length: {len(claims_data)}")

  if isinstance(claims_data_validation, Dataset):
    updated_claims_data = claims_data_validation.add_column('corr_sentence', corr_sentences)
  else:
    print("Error: claims_data is not of type datasets.Dataset")
  return updated_claims_data,missing_keys

#print(len(indexed_sentence_pages))



In [8]:
updated_claims_sentences,keys_issue = mapping_wiki_pages_sentence()
updated_claims_sentences_vs,keys_issue_vs = mapping_wiki_pages_sentence_validation()
print(updated_claims_sentences_vs[:5])

#print(indexed_sentence_pages['Chris_Pérez'])


{'id': [91198, 194462, 137334, 137334, 137334], 'label': ['NOT ENOUGH INFO', 'NOT ENOUGH INFO', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS'], 'claim': ['Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League.', 'Tilda Swinton is a vegan.', 'Fox 2000 Pictures released the film Soul Food.', 'Fox 2000 Pictures released the film Soul Food.', 'Fox 2000 Pictures released the film Soul Food.'], 'evidence_annotation_id': [108548, 227768, 289914, 291259, 293412], 'evidence_id': [-1, -1, 283015, 284217, 285960], 'evidence_wiki_url': ['', '', 'Soul_Food_-LRB-film-RRB-', 'Soul_Food_-LRB-film-RRB-', 'Soul_Food_-LRB-film-RRB-'], 'evidence_sentence_id': [-1, -1, -1, -1, -1], 'corr_sentence': ['', '', '', '', '']}


In [9]:
print(updated_claims_sentences_vs[:5])
#print(type(claims_data)) Checking duplicate records in claims dataset
import pandas as pd
df = pd.DataFrame(updated_claims_sentences)
df_valid = pd.DataFrame(updated_claims_sentences_vs)
#df.head(10)



{'id': [91198, 194462, 137334, 137334, 137334], 'label': ['NOT ENOUGH INFO', 'NOT ENOUGH INFO', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS'], 'claim': ['Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League.', 'Tilda Swinton is a vegan.', 'Fox 2000 Pictures released the film Soul Food.', 'Fox 2000 Pictures released the film Soul Food.', 'Fox 2000 Pictures released the film Soul Food.'], 'evidence_annotation_id': [108548, 227768, 289914, 291259, 293412], 'evidence_id': [-1, -1, 283015, 284217, 285960], 'evidence_wiki_url': ['', '', 'Soul_Food_-LRB-film-RRB-', 'Soul_Food_-LRB-film-RRB-', 'Soul_Food_-LRB-film-RRB-'], 'evidence_sentence_id': [-1, -1, -1, -1, -1], 'corr_sentence': ['', '', '', '', '']}


In [10]:
# Just for understanding data whats actually going on
valid_ids = df[df['label'].isin(['SUPPORTS', 'REFUTES'])].groupby('id').filter(
    lambda g: (g['evidence_sentence_id'].eq(-1).any()) &
              (g['evidence_sentence_id'].nunique() > 1)
)['id'].unique()

# Then filter the original dataframe
result = df[(df['label'].isin(['SUPPORTS', 'REFUTES'])) &
            (df['id'].isin(valid_ids))].sort_values('id')
#result.head(10)

##   Above cases are those in which one is having claim as supported with evidence and other is having claim without evidence_id

results = df.groupby('id').filter(
    lambda group: (
        group['label'].isin(['SUPPORTS', 'REFUTES']).any() and
        (group['evidence_sentence_id'] == -1).any() and
        not (group['evidence_sentence_id'] != -1).any()
    )
)
# checking cases which are completeley useless as
results_valid = df_valid.groupby('id').filter(
    lambda group: (
        group['label'].isin(['SUPPORTS', 'REFUTES']).any() and
        (group['evidence_sentence_id'] == -1).any() and
        not (group['evidence_sentence_id'] != -1).any()
    )
)
'''
# one time check if logic works or not
result1 =  filtered_df.groupby('id').filter(
    lambda group: (
        group['label'].isin(['SUPPORTS', 'REFUTE']).any() and
        (group['evidence_sentence_id'] == -1).any() and
        not (group['evidence_sentence_id'] != -1).any()
    )
)
'''

common_ids = result['id'].isin(results['id']).any()
print(common_ids)
#results
#print(df[df['id']==129629])

False


In [11]:
# Since there are no commonids we need to remove results id from main dataset because they are bit useless

issue_ids = results['id'].unique()
issue_ids_valid = results_valid['id'].unique()

filtered_df = df[~df['id'].isin(issue_ids)]
filtered_df_valid = df_valid[~df_valid['id'].isin(issue_ids_valid)]
filtered_df_valid.head(10)

,id,label,claim,evidence_annotation_id,evidence_id,evidence_wiki_url,evidence_sentence_id,corr_sentence
0,91198,NOT ENOUGH INFO,Colin Kaepernick became a starting quarterback...,108548,-1,,-1,
1,194462,NOT ENOUGH INFO,Tilda Swinton is a vegan.,227768,-1,,-1,
7,166626,NOT ENOUGH INFO,Anne Rice was born in New Jersey.,191656,-1,,-1,
8,166626,NOT ENOUGH INFO,Anne Rice was born in New Jersey.,191657,-1,,-1,
9,111897,REFUTES,Telemundo is a English-language television net...,131371,146144,Telemundo,-1,
10,111897,REFUTES,Telemundo is a English-language television net...,131371,146148,Telemundo,1,It is the second largest provider of Spanish c...
11,111897,REFUTES,Telemundo is a English-language television net...,131371,146150,Telemundo,4,The channel broadcasts programs and original c...
12,111897,REFUTES,Telemundo is a English-language television net...,131371,146150,Hispanic_and_Latino_Americans,-1,
13,111897,REFUTES,Telemundo is a English-language television net...,131371,146151,Telemundo,5,"In addition , Telemundo operates NBC Universo ..."
14,89891,REFUTES,Damon Albarn's debut album was released in 2011.,107201,120581,Damon_Albarn,17,His debut solo studio album Everyday Robots --...


In [12]:
aggregated_df = filtered_df.groupby(['id','label','claim'],as_index=False)['corr_sentence'].apply(' '.join)
aggregated_df.head(10)

aggregated_df_valid = filtered_df_valid.groupby(['id','label','claim'],as_index=False)['corr_sentence'].apply(' '.join)
aggregated_df_valid.head(10)

condition = (
    aggregated_df['label'].isin(['SUPPORTS', 'REFUTES']) &
    (aggregated_df['corr_sentence'] == '')
)

condition_valid = (
    aggregated_df_valid['label'].isin(['SUPPORTS', 'REFUTES']) &
    (aggregated_df_valid['corr_sentence'] == '')
)
matching_records_valid = aggregated_df_valid[condition_valid]
matching_records = aggregated_df[condition]

df[df['id'].isin(matching_records['id']) & ~df['evidence_wiki_url'].isin(keys_issue)]

df_valid[df_valid['id'].isin(matching_records_valid['id']) & ~df_valid['evidence_wiki_url'].isin(keys_issue_vs)]

,id,label,claim,evidence_annotation_id,evidence_id,evidence_wiki_url,evidence_sentence_id,corr_sentence


In [13]:
# time being removing records with keys_issue

special_keys_issue = matching_records['id'].unique()
special_keys_issue_valid = matching_records_valid['id'].unique()
aggregated_df = aggregated_df[~aggregated_df['id'].isin(special_keys_issue)]
aggregated_df_valid = aggregated_df_valid[~aggregated_df_valid['id'].isin(special_keys_issue_valid)]

aggregated_df_valid.size





60276

In [14]:
def tokenize(example, type):
    if(type=='short'):

      return tokenizer(
          example["claim"],
          example["corr_sentence"],
          truncation=True,
          padding="max_length",
          max_length=512
      )
    else:
      chunks = tokenizer(example['claim'],
                         example['corr_sentence'],
                         truncation=True,
                         max_length = 512,
                         stride = 128,
                         return_overflowing_tokens = True,
                         padding = 'max_length')
      return chunks




In [15]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

MAX_LEN = 512
# converting text labels into encoding for model understanding
LABEL_CODING = {'SUPPORTS':0,'REFUTES':1,'NOT ENOUGH INFO':2}
# example is like df
def handling_tokenisation(example):
  tokens = tokenizer(example['claim'],example['corr_sentence'], truncation = False)


  if len(tokens['input_ids'])<=MAX_LEN:
    res =  tokenize(example,'short')
    return [{
        'input_ids': res['input_ids'],
        'attention_mask': res['attention_mask'],
        'labels' : LABEL_CODING[example['label']]
    }]



  else:
    res = tokenize(example,'long')
    # in this case chunks will be returned
    output = []
    for i in range(len(res['input_ids'])):
      output.append({

          'input_ids': res['input_ids'][i],
          'attention_mask': res['attention_mask'][i],
          'labels' : LABEL_CODING[example['label']]
      })

    return output



In [16]:
# applying tokenisation for entire dataframe
tokenized_data = []
tokenized_data_valid = []
for _, row in aggregated_df.iterrows():
  samples = handling_tokenisation(row)
  tokenized_data.extend(samples)

for _, row in aggregated_df_valid.iterrows():
  samples = handling_tokenisation(row)
  tokenized_data_valid.extend(samples)

tokenized_data[0]
tokenized_data_valid[0]

Token indices sequence length is longer than the specified maximum sequence length for this model (699 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': [101,
  2543,
  14876,
  2595,
  2003,
  2019,
  4646,
  1012,
  102,
  2019,
  3176,
  2544,
  1010,
  2543,
  14876,
  2595,
  2005,
  16380,
  1010,
  2001,
  2207,
  1999,
  2397,
  2325,
  1010,
  2021,
  2023,
  2544,
  2515,
  2025,
  2224,
  16216,
  19665,
  2349,
  2000,
  6207,
  1005,
  1055,
  9259,
  14879,
  2353,
  1011,
  2283,
  4773,
  16602,
  2015,
  2000,
  1996,
  4773,
  23615,
  1011,
  2241,
  9621,
  3194,
  2328,
  2046,
  16380,
  1012,
  16380,
  16380,
  16216,
  19665,
  16216,
  19665,
  1006,
  9621,
  3194,
  1007,
  4773,
  23615,
  4773,
  23615,
  9621,
  3194,
  9621,
  3194,
  2543,
  14876,
  2595,
  2005,
  16380,
  2543,
  14876,
  2595,
  2005,
  16380,
  6207,
  6207,
  4297,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [17]:
# convert above created lists into datasets for model evaluation
dataset_train_processed = Dataset.from_list(tokenized_data)
dataset_valid_processed = Dataset.from_list(tokenized_data_valid)



In [18]:
#dataset_train_processed

dataset_valid_processed

Dataset({
    features: ['input_ids', 'attention_mask', 'label_value'],
    num_rows: 15369
})

In [18]:
# starting BERT modeling part
from transformers import BertForSequenceClassification, TrainingArguments, Trainer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)


training_args = TrainingArguments(
    output_dir="./nli_model",                 # Where checkpoints and logs go
    eval_strategy="epoch",             # Evaluate at the end of each epoch
    save_strategy="epoch",                   # Save model at each epoch
    logging_strategy="steps",                # Log metrics every few steps
    logging_steps=200,                       # Log every 200 steps
    learning_rate=2e-5,                      # Good default for BERT
    per_device_train_batch_size=8,          # Depends on your GPU memory (32GB → try 32)
    per_device_eval_batch_size=16,           # Larger batch size for eval is fine
    num_train_epochs=3,                      # 3–5 epochs is usually enough
    weight_decay=0.01,                       # Regularization
    warmup_steps=500,                        # Learning rate warmup
    load_best_model_at_end=True,             # Restore best model after training
    metric_for_best_model="eval_accuracy",   # Use accuracy to pick best checkpoint
    save_total_limit=2,                      # Keep only 2 checkpoints
    fp16=True,                               # Enable mixed precision (if using GPU with support)
    report_to="none",                        # Turn off if not using wandb/tensorboard
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train_processed,  # replace with train split
    eval_dataset=dataset_valid_processed,   # replace with val split
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:



trainer.train()

# Evaluate
results = trainer.evaluate()

Epoch,Training Loss,Validation Loss


In [30]:
# One time handling for special character situation
'''count=0
import unicodedata
import tqdm
#import unicodedata
special_pageids = []
s = 'Mélanie_Laurent'
s = unicodedata.normalize('NFC', s.strip().lower())
#print(s)
#print(filtered_pages[:10])
#print(list(distinct_sentenceids)[:10])
found_ids = [v['id'] for v in d]
for page_id in distinct_sentenceids:
  if page_id not in found_ids:

    special_pageids.append(unicodedata.normalize('NFC', page_id.strip().lower()))


pages = fever_dataset['wikipedia_pages']
# handling part
for page in tqdm.tqdm(pages):
  page_id = str(page.get("id"))
  page_id = unicodedata.normalize('NFC', page_id.strip().lower())
  if page_id in special_pageids and page not in d:
    d.append(page)

  if len(d) == len(distinct_sentenceids):
        break

#print(count)
#print(len(d))
print(len(filtered_pages))
print(len(distinct_sentenceids))
print(keys_issue)
print(claims_data[:10])
print(list(distinct_sentenceids).index('Alexander_Skarsgård'))
found_ids = [v['id'] for v in filtered_pages]
print(len(set(found_ids)))

print(type(claims_data))
print(claims_data[0])

def contains_non_english(word):
    # Check if any character in the word is a non-ASCII character
    return any(unicodedata.category(c) in ['Mn', 'Mc', 'Me'] for c in word)
non_english_words = [word for word in distinct_sentenceids if contains_non_english(word)]
print(len(non_english_words))'''

!pip install --upgrade transformers
import transformers
print(transformers.__version__)

4.52.4
